In [ ]:
#!pip install transformers


In [ ]:
#!pip install -q transformers==4.26.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#!pip install --upgrade pandas

In [ ]:
import pandas as pd

In [ ]:
#!pip install "transformers[sentencepiece]"

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

In [ ]:
from datasets import load_dataset
my_dataset=load_dataset("csv",data_files="/content/drive/MyDrive/doctorado/paper_scientificdata/esp-lsm.csv")

AttributeError: module 'pyarrow.lib' has no attribute 'ListViewType'

In [ ]:
def extract_languages(examples):
  inputs=[ex for ex in examples['esp']]
  targets=[ex for ex in examples['lsm']]
  return {"inputs":inputs , "targets":targets}

In [ ]:
my_dataset=my_dataset.map(extract_languages,batched=True, remove_columns=["esp","lsm"])


In [ ]:
my_dataset=my_dataset.remove_columns(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])


In [ ]:
my_dataset

In [ ]:
my_dataset['train'][0]

In [ ]:
split_dataset=my_dataset['train'].train_test_split(train_size=0.9, seed=42)
split_dataset

In [ ]:
split_dataset['validation']=split_dataset.pop("test")


In [ ]:
split_dataset

In [ ]:
#!pip install datasets

##Create tokenizer

In [ ]:
!huggingface-cli login

In [ ]:
#from transformers import MBartTokenizer
#tokenizer=MBartTokenizer.from_pretrained('facebook/mbart-large-cc25',src_lang='es_XX', tgt_lang='es_XX')


In [ ]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("vgaraujov/bart-base-spanish")
from transformers import AutoModelForSeq2SeqLM,GenerationConfig

model = AutoModelForSeq2SeqLM.from_pretrained("vgaraujov/bart-base-spanish")

translation_generation_config = GenerationConfig(
    num_beams=4,
    early_stopping=True,
    max_length=20,
    decoder_start_token_id=0,
    eos_token_id=model.config.eos_token_id,
    pad_token=model.config.pad_token_id,
)

In [ ]:
#!pip install --upgrade transformers

In [ ]:
sample=split_dataset['train'][10]
sample
inputs=tokenizer(sample['inputs'])
targets=tokenizer(sample['targets'])

print(tokenizer.convert_ids_to_tokens(inputs['input_ids']))
print(tokenizer.convert_ids_to_tokens(targets['input_ids']))

In [ ]:
def preprocess_function(example):

  #if spa_tokenizer.pad_token_id is None:
   # spa_tokenizer.pad_token = "<pad>"
    #spa_tokenizer.pad_token_id = spa_tokenizer.convert_tokens_to_ids(spa_tokenizer.pad_token)
  input=[ex for ex in example['inputs']]
  label=[ex for ex in example['targets']]

  model_inputs=tokenizer(
      input,
      text_target=label,
      max_length=128,
      padding="max_length",
      truncation=True,
      return_tensors="pt"

  )

  return model_inputs

In [ ]:
small_train_dataset=split_dataset['train'].shuffle(seed=42).select(range(1000))
small_eval_dataset=split_dataset['validation'].shuffle(seed=42).select(range(100))

In [ ]:
train_tokenized=small_train_dataset.map(preprocess_function, batched=True)
validation_tokenized=small_eval_dataset.map(preprocess_function, batched=True)
train_tokenized

In [ ]:
train_tokenized_complete=split_dataset['train'].map(preprocess_function, batched=True)
validation_tokenized_complete=split_dataset['validation'].map(preprocess_function, batched=True)

In [ ]:
#batch=data_collator([train_tokenized_complete[i] for i in range(1,4)])
#batch.keys()

In [ ]:
#from transformers import MBartForConditionalGeneration
#model=MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25")

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model=AutoModelForSeq2SeqLM.from_pretrained("vgaraujov/bart-base-spanish")

In [ ]:
#!pip install accelerate -U

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
training_args=Seq2SeqTrainingArguments(
    output_dir="esp-to-lsm-barto-model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_eval_batch_size=16,
    per_device_train_batch_size=32,
    #auto_find_batch_size=True,
    predict_with_generate=True,
    num_train_epochs=30,
    learning_rate=1.5e-5,
    logging_steps=20,
    weight_decay=0.05,
    warmup_steps=300,
    save_total_limit=3,
    overwrite_output_dir=True,
    push_to_hub=True,
    fp16=True, # True if GPU

)


In [ ]:
!pip install rouge_score

In [ ]:
!pip install sacrebleu

In [ ]:
!pip install evaluate

In [ ]:
import evaluate
bleu_metric = evaluate.load("sacrebleu")
ter_metric = evaluate.load("ter")
rouge_metric = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds =tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels,tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    bleu = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    ter_score=ter_metric.compute(predictions=decoded_preds, references=decoded_labels)
    rouge=rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

    return {"bleu": bleu["score"],"rouge":rouge,"ter":ter_score['score']}

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_complete,
    eval_dataset=validation_tokenized_complete,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


In [ ]:
trainer.evaluate(max_length=128)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("esp-to-lsm-model-barto")

In [ ]:
trainer.push_to_hub()

In [ ]:
!pip install latex

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("VaniLara/esp-to-lsm-mbart-model")
model = AutoModelForSeq2SeqLM.from_pretrained("VaniLara/esp-to-lsm-mbart-model")

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="VaniLara/esp-to-lsm-mbart-model")

In [ ]:
!pip install texlive-full

In [ ]:
import pandas as pd
import json

In [ ]:
with open("/content/esp-to-lsm-barto-model/checkpoint-1615/trainer_state.json", 'r') as f:
    data = json.load(f)

In [ ]:
!apt-get install texlive-latex-extra texlive-fonts-recommended dvipng cm-super
!latex --version

In [ ]:
data

In [ ]:
hist=pd.DataFrame(data['log_history'])
hist

hist=hist[['eval_loss','eval_bleu','epoch','loss','eval_rouge','eval_ter']].set_index('epoch')
hist
hist_flattened=pd.json_normalize(hist['eval_rouge'])
hist_result=pd.concat([hist,hist_flattened])
hist_result.drop(columns=['eval_rouge'],inplace=True)
hist_result

In [ ]:
## bleu and ter
metrics=hist_result.drop(columns=['loss','eval_loss','rougeL','rougeLsum','rouge1','rouge2'])
metrics=metrics.dropna()
metrics2=hist_result.drop(columns=['eval_bleu','eval_ter','loss','eval_loss','rougeL','rougeLsum'])
#metrics2=metrics2.dropna()
#metrics2=metrics2[metrics2.index.isin(range(0,21))]
metrics2=metrics2.apply(lambda x: x*100)
metrics.loc[21]=[81.85,8.7]
metrics.reset_index(inplace=True)
#metrics2=metrics2.ffill()
metrics2

In [ ]:
#loss and eval_loss
loss_eval_loss=hist[['eval_loss','loss']]
eval_loss_data=loss_eval_loss['eval_loss'].dropna()
loss_data=loss_eval_loss['loss'].dropna()
data_new=pd.concat([eval_loss_data,loss_data],axis=1)
data_new.columns=['eval_loss','loss']
eval_loss_data

In [ ]:
!which latex
!sudo apt-get install texlive-latex-extra texlive-fonts-recommended dvipng cm-super
!latex --version

In [ ]:
!apt-get update

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
plt.figure(figsize=(15,5))
plt.plot(metrics.index,metrics[['eval_bleu',"eval_ter"]],'--o',label=["BLEU","TER"])
plt.plot(metrics2[['rouge1',"rouge2"]],'--*',label=["ROUGE-1","ROUGE-2"])
plt.xlabel(r'Epoch', fontsize=20)
plt.ylabel(r'Score',fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.grid()



In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
eval_loss_data.plot(y=["eval_loss"],figsize=(10,5))
loss_data.plot(y=["loss"],figsize=(10,5))
plt.xlabel(r'Epoch', fontsize=20)
plt.ylabel(r'Loss',fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15,labels=['validation loss',"train loss"])
plt.grid()
plt.show()

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="VaniLara/esp-to-lsm-barto-model")

In [ ]:
text="  "
tr=pipe(text)
tr

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("VaniLara/esp-to-lsm-barto-model")
model = AutoModelForSeq2SeqLM.from_pretrained("VaniLara/esp-to-lsm-barto-model")